In [1]:
from pyspark.sql import functions as f
from geoCode.geoHash import geoHashSpark as geoHashSpark


In [2]:
countryCode = 'at'

In [3]:
osm = spark.table('dev_sources_osm.cleaned_total_data_set').where(f.lower(f.col('countryCode')) == countryCode).where(f.col('tags').cast('string').contains('building'))
osm.write.mode('overwrite').saveAsTable("data_user_hien." + countryCode + "_osm_building")

In [4]:
buildings = spark.table('data_user_hien.' + countryCode + '_osm_building').withColumn('longitude', f.col('longitude').cast('float')).withColumn('latitude',f.col('latitude').cast('float')).where((f.col('latitude').isNotNull()) &(f.col('longitude').isNotNull()))
pop = spark.table('data_geo.geostat_population_grid_census_lat_lng')
pop = pop.withColumn('lat',f.col('lat').cast('float')).withColumn('lng',f.col('lng').cast('float')).where(f.lower(f.col('CNTR_CODE')) == countryCode).withColumn('TOT_P', f.col('TOT_P').cast('int'))


In [5]:
pop = geoHashSpark(pop,'lat','lng',5)
buildings =geoHashSpark(buildings,'latitude','longitude',5)


In [6]:
buildings.show(n=5)

In [7]:
pop = pop.where(f.col('TOT_P').isNotNull())
buildings = buildings.where(f.col('id').isNotNull())
hashPop = pop.groupBy('geoHash').agg(f.sum(f.col('TOT_P')).alias('popPerHash'))
hashBuilding = buildings.groupBy('geoHash').agg(f.count(f.col('id')).alias('buildPerHash'))


In [8]:
hashPop.count()

In [9]:
hashData = hashPop.alias('pop').join(hashBuilding.alias('build'),hashPop.geoHash == hashBuilding.geoHash, how = 'left_outer').select("pop.*", "build.buildPerHash")
hashData.write.mode('overwrite').saveAsTable('data_user_hien.' + countryCode + '_geo_data')

In [10]:
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import Bucketizer

def decile(inCol,outCol,df):
  discretizer = QuantileDiscretizer(numBuckets=10, inputCol=inCol,outputCol=outCol)  
  dfOut = discretizer.fit(df).transform(df)
  return dfOut

df = decile('popPerHash','resultPop',df)
df = decile('buildPerHash','resultBuild',df)


display(df)

In [11]:
display(df.where((f.col('resultPop') == 0)).where(f.col('resultBuild').isNotNull()).sort(f.col('resultBuild').desc()))

In [12]:
display(df.where(f.col('resultBuild') == 0).groupBy('resultPop').agg(f.count(f.col('resultBuild')).alias('count')).sort(['count'],descending = True))

In [13]:
display(df.where(f.col('resultPop') == 0).groupBy('resultBuild').agg(f.count(f.col('resultPop')).alias('count')).sort(['count'],descending = True))

In [14]:
df.count()

In [15]:
display(df.where(f.col('resultPop') == 9).groupBy('resultBuild').agg(f.count(f.col('resultPop')).alias('count')).sort(['count'],descending = False))

In [16]:
display(df.where(f.col('resultBuild') == 9).groupBy('resultPop').agg(f.count(f.col('resultBuild')).alias('count')).sort(['count'],descending = False))